In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import os, sys, random, ast

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import datasets
from datasets import Dataset, DatasetDict

import torch

from dataclasses import dataclass

from transformers import AutoTokenizer, AutoModelForMultipleChoice
from transformers import Trainer, TrainingArguments, set_seed
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

from typing import Optional, Union

!git lfs install

Git LFS initialized.


In [6]:
# Function to read our JSON file
def read_json(folder_path):
    dataset = []
    for item in os.listdir(folder_path):
        with open(os.path.join(folder_path, item), 'r') as file:
            data = json.load(file)
            dataset.extend(data)
    return dataset

In [7]:
# Load and preprocess training data
train_dataset_path = '/content/drive/MyDrive/NLP/Project/data/training_dataset'

train_data = read_json(train_dataset_path)

In [8]:
# Load and preprocess testing data
test_dataset_path = '/content/drive/MyDrive/NLP/Project/data/validation_dataset'

test_data = read_json(test_dataset_path)

In [9]:
train_df = pd.DataFrame(train_data)

In [10]:
test_df = pd.DataFrame(test_data)

In [11]:
train_df.head()

,Story,Question,Answer Choices,Answer,Reasoning
0,"During 1947, after World War II, the infamous ...",When did the partition of India spark religiou...,"[1939, 1947, 1950, 1962]",1947,The story mentions that the partition of India...
1,"During 1947, after World War II, the infamous ...",Who attempted to mitigate religious violence d...,"[Bhagat Singh, Winston Churchill, Rajendra Pra...",Mahatma Gandhi,"According to the story, during the partition, ..."
2,"During 1947, after World War II, the infamous ...",Which provinces in India experienced most of t...,"[Kerala and Tamil Nadu, Maharashtra and Karnat...",Punjab and Bengal,The story mentions that the most of the religi...
3,"During 1947, after World War II, the infamous ...",When did the religious violence sparked by the...,"[1945, 1947, 1948, 1952]",1948,The story states that the religious violence s...
4,"During 1947, after World War II, the infamous ...",Which new country was created as a result of I...,"[Bangladesh, China, Sri Lanka, Pakistan]",Pakistan,The story mentions that Pakistan was created d...


In [12]:
test_df.head()

,Story,Question,Answer Choices,Answer,Reasoning
0,"In February, Ramesh, a painter from Delhi, tra...",How much time did Ramesh spend in Uttar Prades...,"[Less than a month, About a month, More than a...",About a month,Ramesh travelled to Uttar Pradesh in February ...
1,"In February, Ramesh, a painter from Delhi, tra...",If Ramesh wanted to visit a friend in Guwahati...,"[25th May, 28th May, 1st June, 5th June]",28th May,The Ziro Festival in Arunachal Pradesh ended o...
2,"In February, Ramesh, a painter from Delhi, tra...",Approximately how many months did Ramesh spend...,"[4 months, 5 months, 6 months, 7 months]",4 months,Ramesh first travelled from Delhi to Uttar Pra...
3,"In February, Ramesh, a painter from Delhi, tra...",If Ramesh had to be in Delhi within three days...,"[27th May, 28th May, 29th May, 30th May]",27th May,Since Ramesh had to be in Delhi within three d...
4,"In February, Ramesh, a painter from Delhi, tra...","Where was Ramesh, in the month of April?","[Delhi, Uttar Pradesh, Agra, Arunachal Pradesh]",Delhi,"According to the story, after Ramesh returns f..."


In [13]:
traindata = pd.DataFrame()

In [14]:
answer0,answer1,answer2,answer3 = [],[],[],[]
labels = []
story = []
question = []
for item in train_df.iterrows():
    if len(item[1]['Answer Choices'])==4:
        answer0.append(item[1]['Answer Choices'][0])
        answer1.append(item[1]['Answer Choices'][1])
        answer2.append(item[1]['Answer Choices'][2])
        answer3.append(item[1]['Answer Choices'][3])
        choice = item[1]['Answer Choices']
        answer = item[1]['Answer']
        idx = choice.index(answer)
        labels.append(idx)
        question.append(item[1]['Question'])
        story.append(item[1]['Story'])

traindata['Story'] = story
traindata['Question'] = question
traindata['answer0'] = answer0
traindata['answer1'] = answer1
traindata['answer2'] = answer2
traindata['answer3'] = answer3
traindata['labels'] = labels

In [15]:
traindata.sample(5)

,Story,Question,answer0,answer1,answer2,answer3,labels
2257,The city of Stavanger in Norway has always att...,What was the impact on Stavanger's population ...,Decreased by 10%,Increased by 20%,Increased by 10%,Stabilized,1
1419,"In June 2019, an enthralling film production e...",When did the film project in Shanghai begin?,June 2019,September 2021,January 2023,None of the above,2
386,"In 2010, a wave of revolutions started in Iran...",In which month did the protests begin in Brazil?,June,July,August,September,2
126,"During the 16th century, the Mughal Empire, le...",In which city is the Taj Mahal located?,Delhi,Jaipur,Agra,Lahore,2
294,"In the 18th century, the American colonies sou...",When did the American Revolution begin?,1492,1607,1775,1947,2


In [16]:
testdata = pd.DataFrame()

In [17]:
answer0,answer1,answer2,answer3 = [],[],[],[]
labels = []
story = []
question = []
for item in test_df.iterrows():
    if len(item[1]['Answer Choices'])==4:
        answer0.append(item[1]['Answer Choices'][0])
        answer1.append(item[1]['Answer Choices'][1])
        answer2.append(item[1]['Answer Choices'][2])
        answer3.append(item[1]['Answer Choices'][3])
        choice = item[1]['Answer Choices']
        answer = item[1]['Answer']
        idx = choice.index(answer)
        labels.append(idx)
        question.append(item[1]['Question'])
        story.append(item[1]['Story'])

testdata['Story'] = story
testdata['Question'] = question
testdata['answer0'] = answer0
testdata['answer1'] = answer1
testdata['answer2'] = answer2
testdata['answer3'] = answer3
testdata['labels'] = labels

In [18]:
testdata.sample(5)

,Story,Question,answer0,answer1,answer2,answer3,labels
119,"In the early 1900s, Australia found herself am...",What was the focus of the social revolutions i...,Voting rights,Land rights,Better wages and working conditions,Environmental issues,3
138,"In April of 2027, Jack and his friends decided...",When did Jack and his friends return home from...,April 7th,April 9th,April 8th,April 10th,2
47,"In 1999, Bob, a member of the Liberal Party, r...",In which year did Bob possibly stay out of pol...,1999,2004,2005,2011,2
136,"Grace, a travel blogger, decided to attend a c...",When did Grace return home from the festival?,March 8th,March 10th,March 15th,March 20th,1
156,"Every year in December, the Australian town of...",On which date was the friendly cooking competi...,"December 5, 2020","December 6, 2020","December 7, 2020","December 8, 2020",1


In [19]:
# Split the DataFrame into train_df and val_df
train_df = traindata
test_df = testdata

# Reset the index for both DataFrames (optional)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


In [20]:
print(len(train_df), len(test_df))

2400 240


In [21]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [22]:
ds = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
})

print("Training Dataset Shape:", ds['train'].shape)
print("Testing Dataset Shape:", ds['test'].shape)

print(ds['test'].features)

Training Dataset Shape: (2400, 7)
Testing Dataset Shape: (240, 7)
{'Story': Value(dtype='string', id=None), 'Question': Value(dtype='string', id=None), 'answer0': Value(dtype='string', id=None), 'answer1': Value(dtype='string', id=None), 'answer2': Value(dtype='string', id=None), 'answer3': Value(dtype='string', id=None), 'labels': Value(dtype='int64', id=None)}


In [23]:
def show_one_example(sample):
    print(f"Context: {sample['Story']} {sample['Question']}")
    print(f" 0 - {sample['answer0']}")
    print(f" 1 - {sample['answer1']}")
    print(f" 2 - {sample['answer2']}")
    print(f" 3 - {sample['answer3']}")
    print(f"\nGround Truth: option {sample['labels']}")

In [24]:
show_one_example(ds['train'][2])

Context: During 1947, after World War II, the infamous partition of India occurred, leading to the creation of Pakistan. The decision sparked widespread religious violence, especially in the provinces of Punjab and Bengal. Simultaneously, Mahatma Gandhi, known for his peaceful protests, tried to mitigate the violence with his speeches. By the time the violence ended in 1948, millions were displaced and the demographic structure of the subcontinent was altered. The effects of this event can still be seen today in India and Pakistan's tumultuous relationship. Which provinces in India experienced most of the religious violence during the partition?
 0 - Kerala and Tamil Nadu
 1 - Maharashtra and Karnataka
 2 - Punjab and Bengal
 3 - Assam and Meghalaya

Ground Truth: option 2


In [25]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [26]:
ending_names = ["answer0", "answer1", "answer2", "answer3"]

def preprocess_function(samples):
    first_sentences = [[context] * len(ending_names) for context in samples['Story']]

    question_headers = samples['Question']
    second_sentences = [[f"{header} {samples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_samples = tokenizer(first_sentences, second_sentences, truncation=True)

    return {k: [v[i:i + len(ending_names)] for i in range(0, len(v), len(ending_names))] for k, v in tokenized_samples.items()}

In [27]:
encoded_ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [28]:
encoded_ds['train'].features

{'Story': Value(dtype='string', id=None),
 'Question': Value(dtype='string', id=None),
 'answer0': Value(dtype='string', id=None),
 'answer1': Value(dtype='string', id=None),
 'answer2': Value(dtype='string', id=None),
 'answer3': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), length=-1, id=None)}

In [29]:
encoded_ds['test'].features

{'Story': Value(dtype='string', id=None),
 'Question': Value(dtype='string', id=None),
 'answer0': Value(dtype='string', id=None),
 'answer1': Value(dtype='string', id=None),
 'answer2': Value(dtype='string', id=None),
 'answer3': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), length=-1, id=None)}

In [30]:
model = AutoModelForMultipleChoice.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad inputs
    for the multiple choices received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()}
                               for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Unflatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [32]:
def compute_metrics(predictions):
    preds, labels = predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == labels).astype(np.float32).mean().item()}

In [33]:
MODEL_CKPT = 'bert-base-uncased'
MODEL_NAME = MODEL_CKPT.split("/")[-1] + "-MCQ_Task"

STRATEGY = "epoch"
LEARNING_RATE = 5e-5
BATCH_SIZE = 16
NUM_OF_EPOCHS = 5
MAX_LENGTH = 512
WEIGHT_DECAY = 0.01
REPORTS_TO = "tensorboard"

set_seed = 42

In [34]:
args = TrainingArguments(
    output_dir='bert-base-uncased-MCQ_Task',
    evaluation_strategy="epoch",
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_first_step=True,
)

In [35]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

In [36]:
train_results = trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.393000,1.134565,0.500000
2,1.087900,1.063718,0.537500
3,0.715200,1.228522,0.525000
4,0.452800,1.166498,0.566667
5,0.291500,1.338205,0.566667


In [37]:
metrics = trainer.evaluate()

{'eval_loss': 1.3382049798965454, 'eval_accuracy': 0.5666666626930237, 'eval_runtime': 9.0237, 'eval_samples_per_second': 26.597, 'eval_steps_per_second': 4.433, 'epoch': 5.0}


In [38]:
metrics

{'eval_loss': 1.3382049798965454,
 'eval_accuracy': 0.5666666626930237,
 'eval_runtime': 9.0237,
 'eval_samples_per_second': 26.597,
 'eval_steps_per_second': 4.433,
 'epoch': 5.0}